
# Optimización de Precios y Descuentos Personalizados - Proyecto Gym Master

## Objetivo
Simular la elasticidad al precio en base a los perfiles de pago de los socios, para proponer estrategias de descuento que minimicen el churn y aumenten los ingresos.
Trabajaremos con los segmentos generados previamente: Puntuales, Retraso leve y Morosos crónicos.


In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Configuración de gráficos
sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (12,6)

# Cargar el dataset de pagos con segmento incluido
pagos_df = pd.read_csv('../Data_Lake/pagos_simulados.csv')
segmentos_df = pd.read_csv('../Data_Lake/segmentacion_socios.csv')  # asumimos que se guarda desde el notebook anterior

# Unir el segmento al dataset de pagos
pagos_df = pagos_df.merge(segmentos_df[['socio_id', 'segmento_pago']], on='socio_id')
pagos_df.head()


In [ ]:

# Simulación de elasticidad al precio
precios_actuales = {'Básico': 30, 'Estándar': 50, 'Premium': 70}
variaciones_precio = np.arange(-0.3, 0.31, 0.1)  # de -30% a +30%

# Simulamos probabilidad de pago por variación de precio y segmento
elasticidad = {
    'Puntual': -0.2,        # baja sensibilidad
    'Retraso leve': -0.5,   # sensibilidad media
    'Moroso crónico': -0.8  # muy sensible al precio
}

resultados = []
for nivel, precio_base in precios_actuales.items():
    for var in variaciones_precio:
        precio_nuevo = precio_base * (1 + var)
        for segmento, sens in elasticidad.items():
            prob_pago = max(0, 1 + sens * var)
            ingresos_esperados = precio_nuevo * prob_pago
            resultados.append({
                'nivel': nivel,
                'segmento': segmento,
                'variacion_precio': round(var,2),
                'precio_nuevo': round(precio_nuevo,2),
                'prob_pago': round(prob_pago,2),
                'ingresos_esperados': round(ingresos_esperados,2)
            })

df_elasticidad = pd.DataFrame(resultados)
df_elasticidad.head()


In [ ]:

# Visualización de la elasticidad
sns.lineplot(data=df_elasticidad, x='variacion_precio', y='ingresos_esperados', hue='segmento', style='nivel')
plt.title('Elasticidad al Precio por Segmento y Nivel de Subscripción')
plt.xlabel('Variación de Precio (%)')
plt.ylabel('Ingresos Esperados')
plt.show()


In [ ]:

# Propuesta de descuentos personalizados
propuesta_descuento = pd.DataFrame({
    'segmento': ['Puntual', 'Retraso leve', 'Moroso crónico'],
    'descuento_sugerido': ['5%', '10%', '20%'],
    'justificacion': [
        'Incentivo pequeño o beneficios premium',
        'Reducir riesgo de atraso incentivando puntualidad',
        'Reducir barrera de pago para minimizar churn'
    ]
})
propuesta_descuento



## Conclusión

- Los socios **puntuales** mantienen alta probabilidad de pago incluso con aumentos leves de precio.
- Los **retraso leve** y **morosos crónicos** muestran mayor caída en ingresos si el precio sube.
- Por ello, proponemos aplicar **descuentos diferenciados** según el segmento para optimizar ingresos y retención.

Esta estrategia puede implementarse junto con un sistema de alertas para identificar cambios de comportamiento.
